# Clustering Crypto

In [2]:
# Initial imports
import requests
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [3]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [4]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [38]:
# Create a DataFrame 

# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

### Data Preprocessing

In [33]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
new_crypto_df = crypto_df.drop(columns = ["Unnamed: 0"])
new_crypto_df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1247,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
1251,SteamPunk,PoS,False,PoS,NaN,40000000


In [34]:
# Keep only cryptocurrencies that are trading
new_crypto_df = new_crypto_df.loc[new_crypto_df["IsTrading"] == True]
new_crypto_df.tail(20)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1225,uPlexa,CryptoNight,True,PoW,NaN,10500000000
1226,TigerCash,SHA-256,True,PoS,1.000000e+09,1000000000
1228,Tip Blockchain,DPoS,True,PoS,NaN,1000000000
1229,DECOIN,X11,True,PoS,NaN,140000000
1230,Waves,Leased POS,True,LPoS,1.000000e+08,100000000
1231,Particl,PoS,True,PoS,9.283138e+06,8634140
1232,Slate,Slatechain,True,PoW,NaN,950000000
1234,BitTorrent,TRC10,True,DPoS,9.899887e+11,990000000000
1235,XAYA,NeoScrypt,True,PoW,NaN,77303932
1237,Nxt,PoS,True,PoS/LPoS,1.000000e+09,1000000000


In [40]:
# Keep only cryptocurrencies with a working algorithm
new_crypto_df = new_crypto_df.loc[new_crypto_df["Algorithm"] != "N/A"]

In [41]:
# Remove the "IsTrading" column
new_crypto_df = new_crypto_df.drop(columns = "IsTrading")

In [96]:
# Remove rows with at least 1 null value
new_crypto_df=new_crypto_df.dropna()
new_crypto_df.isnull().sum()

CoinName           0
Algorithm          0
IsTrading          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [42]:
# Remove rows with cryptocurrencies having no coins mined
new_crypto_df = new_crypto_df.loc[new_crypto_df["TotalCoinsMined"] != 0]

In [43]:
# Drop rows where there are 'N/A' text values
new_crypto_df = new_crypto_df.loc[new_crypto_df["TotalCoinSupply"].str.contains("N/A") != True]
new_crypto_df = new_crypto_df.loc[new_crypto_df["ProofType"].str.contains("N/A") != True]


In [44]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name = new_crypto_df["CoinName"]

In [46]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
new_crypto_df = new_crypto_df.drop(columns = "CoinName")

In [ ]:
# Create dummy variables for text features


In [ ]:
# Standardize data


### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components


In [ ]:
# Create a DataFrame with the principal components data


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [84]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(new_crypto_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

ValueError: could not convert string to float: 'PoS'

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
